In this notebook, we will look at the frequency distribution of power in responses to stimuli in different brain states.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, patches
import matplotlib.gridspec as gridspec
from scipy import signal
import umap
import mne
from mne.preprocessing import compute_current_source_density
from mne.connectivity import spectral_connectivity
from mne import EpochsArray

from tbd_eeg.data_analysis.eegutils import *
from tbd_eeg.data_analysis.Utilities import utilities as utils

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False
%matplotlib widget

In [2]:
epoch_cms = {
    'pre' : cm.Reds,
    'iso_high' : cm.PuOr,
    'iso_low' : cm.PuOr_r,
    'early_recovery': cm.Blues,
    'late_recovery' : cm.Greens
}

In [3]:
data_folder = "../tiny-blue-dot/zap-n-zip/EEG_exp/mouse521886/estim1_2020-07-16_13-37-02/experiment1/recording1/"

# set the sample_rate for all data analysis
sample_rate = 2500

# load experiment metadata and eeg data
exp = EEGexp(data_folder)
eegdata = exp.load_eegdata(frequency=sample_rate, return_type='pd')

# locate valid channels (some channels can be disconnected and we want to ignore them in the analysis)
print('Identifying valid channels...')
median_amplitude = eegdata[:sample_rate*300].apply(
    utils.median_amplitude, raw=True, axis=0, distance=sample_rate
)
valid_channels = median_amplitude.index[median_amplitude < 2000].values
print('The following channels seem to be correctly connected and report valid data:')
print(list(valid_channels))

# load other data (running, iso etc)
print('Loading other data...')
running_speed = exp.load_running(return_type='pd')
iso = exp.load_analog_iso(return_type='pd')

# automatically annotate anesthesia epochs
iso_first_on = (iso>4).idxmax()
print('iso on at', iso_first_on)
iso_first_mid = ((iso[iso.index>iso_first_on]>1)&(iso[iso.index>iso_first_on]<4)).idxmax()
print('iso reduced at', iso_first_mid)
iso_first_off = (iso>1)[::-1].idxmax()
print('iso off at', iso_first_off)

# annotate artifacts with power in high frequencies
print('Annotating artifacts...')
hf_annots = pd.Series(
    eegdata[valid_channels].apply(
        find_hf_annotations, axis=0,
        sample_rate=sample_rate, fmin=300, pmin=0.25
    ).mean(axis=1),
    name='artifact'
)
recovery_first_jump = (hf_annots>4)[hf_annots.index>iso_first_off].idxmax()

epochs = pd.Series(
    index = [0, iso_first_on-0.001, iso_first_on+0.001, iso_first_mid-0.001,
             iso_first_mid+0.001, iso_first_off-0.001, iso_first_off+0.001,
             recovery_first_jump-0.001, recovery_first_jump+0.001, eegdata.index[-1]],
    data=['pre', 'pre', 'iso_high', 'iso_high', 'iso_low', 'iso_low',
          'early_recovery', 'early_recovery', 'late_recovery', 'late_recovery'],
    dtype=pd.CategoricalDtype(
        categories=['pre', 'iso_high', 'iso_low', 'early_recovery', 'late_recovery'],
        ordered=True
    )
)

No *stim* files were found. Experiment type: spontaneous.
Identifying valid channels...
The following channels seem to be correctly connected and report valid data:
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Loading other data...


/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:454: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return eval(self.dfile['analog_meta'].value)


iso on at 644.11
iso reduced at 1190.05
iso off at 1867.56
Annotating artifacts...


In [20]:
if exp.experiment_type == 'electrical stimulation':
    stimuli = pd.read_csv(exp.stimulus_log_file)
else:
    stimuli = None
stimuli

,stim_type,amplitude,duration,onset,offset,sweep
0,biphasic,50,400.0,134.81231,134.81291,0
1,biphasic,20,400.0,138.40947,138.41007,0
2,biphasic,50,400.0,142.72804,142.72864,0
3,biphasic,100,400.0,147.04601,147.04661,0
4,biphasic,100,400.0,151.26887,151.26948,0
...,...,...,...,...,...,...
895,biphasic,20,400.0,4168.44474,4168.44535,2
896,biphasic,100,400.0,4172.79589,4172.79649,2
897,biphasic,100,400.0,4176.85687,4176.85748,2
898,biphasic,100,400.0,4181.28689,4181.28750,2


In [4]:
# validate data and artifact annotation
f, ax = plt.subplots(1, 1, figsize=(12, 2), tight_layout=True)
eegdata[[13, 4]][::20].plot(ax=ax, alpha=0.7)
ax2 = ax.twinx()
hf_annots.plot(ax=ax2, c='r', lw=0.5)
ax2.set_ylim(-np.sum(ax2.get_ylim())/2, ax2.get_ylim()[1])
for si in stimuli[stimuli.amplitude==20].onset:
    ax.axvline(si, c=cm.Greys(0.3, 0.3))
for si in stimuli[stimuli.amplitude==50].onset:
    ax.axvline(si, c=cm.Greys(0.6, 0.3))
for si in stimuli[stimuli.amplitude==100].onset:
    ax.axvline(si, c=cm.Greys(0.9, 0.3));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'stimuli' is not defined

# preprocessing

In [5]:
egroups = {
    'left_front' : [11, 12, 13, 14],
    'right_front' : [18, 17, 16, 15],
    'left_front_middle' : [9, 10],
    'right_front_middle' : [20, 19],
    'left_back_middle' : [3, 4, 6, 7],
    'right_back_middle' : [26, 25, 23, 22],
    'left_back_middle_center' : [5, 8],
    'right_back_middle_center' : [24, 21],
    'left_back' : [1, 2],
    'right_back' : [28, 27],
    'left_bottom' : [0],
    'right_bottom' : [29],
}

exp.ch_coordinates['z'] = 0
exp.ch_coordinates['group'] = ''
exp.ch_coordinates['gid'] = 0
exp.ch_coordinates['wgid'] = 0
for i, (g, idx) in enumerate(egroups.items()):
    exp.ch_coordinates.loc[idx, 'group'] = g
    exp.ch_coordinates.loc[idx, 'gid'] = i
    exp.ch_coordinates.loc[sorted(idx), 'wgid'] = idx
exp.ch_coordinates = exp.ch_coordinates.sort_values(['gid', 'wgid'])
exp.ch_coordinates['order'] = 0
_left = exp.ch_coordinates.index[exp.ch_coordinates.group.str.contains('left')]
exp.ch_coordinates.loc[_left, 'order'] = range(len(_left))
_right = exp.ch_coordinates.index[exp.ch_coordinates.group.str.contains('right')]
exp.ch_coordinates.loc[_right, 'order'] = len(_left)+np.arange(len(_right))[::-1]
exp.ch_coordinates.sort_index(inplace=True)
exp.ch_coordinates.drop('wgid', inplace=True, axis=1)

In [6]:
# define a function to quickly plot the electrode map with or without borders
def plot_electrode_map(ax, highlight=None, labels=True, cmap=cm.Paired, s=50):
    colors = np.array(exp.ch_coordinates.gid.map(lambda x: cmap(x/11, 0.9)))
    if highlight in set(exp.ch_coordinates.group):
        colors = np.array(exp.ch_coordinates.apply(lambda row: cmap(row.gid/12, 0.9) if row.group==highlight else cm.Greys(0.5,0.5), axis=1))
    exp.ch_coordinates.plot(
        kind='scatter', x='ML', y='AP', marker='o', ax=ax, legend=False, c=colors, s=s
    )
    if labels:
        for i in exp.ch_coordinates.index:
            ax.annotate(exp.ch_coordinates.loc[i, 'order'], exp.ch_coordinates.loc[i, ['AP', 'ML']][::-1]+[0, 0.2], xycoords='data', ha='center')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 3)
    if labels:
        ax.set_title('Electrode map')
    if not labels:
        ax.set_xlabel('')
        ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([]);

# function to show electrode groups along an axis instead of electrode numbers
def draw_groups(ax, cmap=cm.Paired):
    ax.set_xlim(-0.5, 29.5)
    ax.set_ylim(29.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    nt = 0
    def add_patch(df):
        nonlocal nt, ax
        ax.add_patch(patches.Rectangle(
            (nt+0.05-0.5, 30), len(df)-0.05, 1, clip_on=False, color=cmap(df.gid.iloc[0]/12, 0.9), label=df.group.iloc[0]
        ))
        ax.add_patch(patches.Rectangle(
            (-2, nt+0.05-0.5), 1, len(df), clip_on=False, color=cmap(df.gid.iloc[0]/12, 0.9)
        ))
        nt += len(df)
    exp.ch_coordinates.sort_values('order').groupby('gid', sort=False).apply(add_patch)

In [7]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=False)
plot_electrode_map(ax, labels=True, s=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

# Responses

In [21]:
stimuli['state'] = ''
stimuli.set_index('onset', inplace=True)
stimuli.loc[:iso_first_on, 'state'] = 'awake'
stimuli.loc[iso_first_on:iso_first_mid, 'state'] = 'induction'
stimuli.loc[iso_first_mid:iso_first_off, 'state'] = 'anesthetized'
stimuli.loc[iso_first_off:, 'state'] = 'recovery'
stimuli

,stim_type,amplitude,duration,offset,sweep,state
onset,,,,,,
134.81231,biphasic,50,400.0,134.81291,0,awake
138.40947,biphasic,20,400.0,138.41007,0,awake
142.72804,biphasic,50,400.0,142.72864,0,awake
147.04601,biphasic,100,400.0,147.04661,0,awake
151.26887,biphasic,100,400.0,151.26948,0,awake
...,...,...,...,...,...,...
4168.44474,biphasic,20,400.0,4168.44535,2,recovery
4172.79589,biphasic,100,400.0,4172.79649,2,recovery
4176.85687,biphasic,100,400.0,4176.85748,2,recovery
